In [ ]:
##This is the extension of 4-ClientTest.py in notebook, but runninbg multiple tests using goldenset stored in the deepeval portal

In [1]:
import os
from dotenv import load_dotenv
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval import evaluate
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import AzureCliCredential

# Load environment variables
load_dotenv()

True

In [2]:
# Initialize Azure AI Project Client
def get_project_client():
    endpoint = os.environ.get("AZURE_AI_PROJECT_ENDPOINT")
    if not endpoint:
        raise ValueError("AZURE_AI_PROJECT_ENDPOINT not set in environment.")
    return AIProjectClient(endpoint=endpoint, credential=AzureCliCredential())

# Get agent by ID
def get_agent(client, agent_id):
    agent = client.agents.get_agent(agent_id)
    if not agent:
        raise ValueError(f"Agent with ID {agent_id} not found.")
    return agent

# Run agent thread and get response
def run_agent_thread(client, agent_id, user_message):
    final_run = client.agents.create_thread_and_process_run(
        agent_id=agent_id,
        thread={
            "messages": [
                {"role": "user", "content": user_message}
            ]
        },
    )
    if final_run.status != "completed":
        raise RuntimeError("Agent run did not complete successfully.")
    messages = client.agents.messages.list(thread_id=final_run.thread_id, order=ListSortOrder.ASCENDING)
    for message in messages:
        if message.run_id == final_run.id and message.text_messages:
            return message.text_messages[-1].text.value
    return ""

In [3]:
client = get_project_client()
agent_id = "asst_s9vGyZLmHIj49kSz3Z05sti5"
#user_question = "The dog chased the cat up the tree, who ran up the tree?"
agent = get_agent(client, agent_id)
#response = run_agent_thread(client, agent.id, user_question)

In [4]:
from deepeval.dataset import EvaluationDataset
dataset = EvaluationDataset()
dataset.pull(alias="sample")
for golden in dataset.goldens:
    test_case = LLMTestCase(
        input=golden.input,
        actual_output=run_agent_thread(client, agent.id, golden.input)
    )
    dataset.add_test_case(test_case)

/workspaces/AgentTesting/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [5]:
from deepeval.metrics import AnswerRelevancyMetric
evaluate(test_cases=dataset.test_cases, metrics=[AnswerRelevancyMetric()])

✨ You're running DeepEval's latest Answer Relevancy Metric! (using Azure OpenAI (gpt-4.1), strict=False, 
async_mode=True)...



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: Azure OpenAI (gpt-4.1), reason: The score is 1.00 because the answer was perfectly relevant and directly addressed the question with no irrelevant information. Great job!, error: None)

For test case:

  - input: Who wrote 'Hamlet'?
  - actual output: **'Hamlet'** was written by **William Shakespeare**.
  - expected output: None
  - context: None
  - retrieval context: None


Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: Azure OpenAI (gpt-4.1), reason: The score is 1.00 because the answer was perfectly relevant and directly addressed the question with no irrelevant information. Great job!, error: None)

For test case:

  - input: What is the chemical symbol for water?
  - actual output: The chemical symbol for water is **H₂O**.
  - expected output: None
  - context: None
  - retrieval context: None


Metrics Summary

  - ✅ Answe

✓ Done 🎉! View results on 
]8;id=968225;https://app.confident-ai.com/project/cmezc8wwy041vzt5726njpftj/evaluation/test-runs/cmfg2crqo03hc8d18rdqsouf4/compare-test-results\https://app.confident-ai.com/project/cmezc8wwy041vzt5726njpftj/evaluation/test-runs/cmfg2crqo03hc8d18rdqsouf4/compa]8;;\
]8;id=968225;https://app.confident-ai.com/project/cmezc8wwy041vzt5726njpftj/evaluation/test-runs/cmfg2crqo03hc8d18rdqsouf4/compare-test-results\re-test-results]8;;\

EvaluationResult(test_results=[TestResult(name='test_case_1', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because the answer was perfectly relevant and directly addressed the question with no irrelevant information. Great job!', strict_mode=False, evaluation_model='Azure OpenAI (gpt-4.1)', error=None, evaluation_cost=0.00317, verbose_logs='Statements:\n[\n    "\'Hamlet\' was written by William Shakespeare."\n] \n \nVerdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": null\n    }\n]')], conversational=False, multimodal=False, input="Who wrote 'Hamlet'?", actual_output="**'Hamlet'** was written by **William Shakespeare**.", expected_output=None, context=None, retrieval_context=None, additional_metadata=None), TestResult(name='test_case_2', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because the answer was 